In [1]:
import numpy as np
n= 5
m = 5
num_states = n * m  # 25 cases


S = np.arange(num_states)
A = np.array([0, 1, 2, 3])  # 0: left, 1 : right, 2: top, 3: bot
T = np.array([2, 11, 24])  # Terminal State ( position)
P = np.zeros((len(S), len(A), len(S), 2))

for x in range(n * m):
    if (x + 1) % n != 0:  # Droite
        P[x, 1, x + 1, 0] = 1.0
    if x % n != 0:  # Gauche
        P[x, 0, x - 1, 0] = 1.0
    if x >= n:  # Top
        P[x, 2, x - n, 0] = 1.0
    if x < n * m - n:  # Bot
        P[x, 3, x + n, 0] = 1.0
print(S)
        
# rewards

P[1, 1, 2, 1] = -1
P[3, 0, 2, 1] = -1
P[7, 2, 2, 1] = -1

P[6, 3, 11, 1] = -1
P[10, 1, 11, 1] = -1
P[12, 0, 11, 1] = -1
P[16, 2, 11, 1] = -1

# P[13, 3, 18, 1] = -1
# P[17, 1, 11, 1] = -1
# P[19, 0, 18, 1] = -1
# P[23, 2, 18, 1] = -1

P[23, 1, 24, 1] = 1
P[19, 3, 24, 1] = 1

def reset() -> int:
    return n * m  // 2


def is_terminal(state: int) -> bool:
    return state in T


def step(state: int, a: int) -> (int, float, bool):
    assert (state not in T)
    s_p = np.random.choice(S, p=P[state, a, :, 0]) # trouve ta position d'arrivée
    r = P[state, a, s_p, 1] # recupere reward
    return s_p, r, (s_p in T) # position, reward, si terminal

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


In [11]:
def iterative_policy_evaluation(
        S: np.ndarray,
        A: np.ndarray,
        P: np.ndarray,
        T: np.ndarray,
        Pi: np.ndarray,
        gamma: float = 0.99,
        theta: float = 0.0001, # accuracy
        V: np.ndarray = None
) -> np.ndarray:
    assert 0 <= gamma <= 1
    assert theta > 0
    if V is None:
        V = np.random.random((S.shape[0],))
        V[T] = 0.0
    while True:
        delta = 0
        for s in S:
            v_temp = V[s]
            tmp_sum = 0
            for a in A:
                for s_p in S: # proba x faisabilité x (reward + longterme x Value s')
                    tmp_sum += Pi[s, a] * P[s, a, s_p, 0] * (
                            P[s, a, s_p, 1] + gamma * V[s_p]
                    )
            V[s] = tmp_sum
            delta = np.maximum(delta, np.abs(tmp_sum - v_temp))
        if delta < theta:
            break
    return V

In [12]:
def tabular_uniform_random_policy(space_size: int, action_size: int):
    return np.ones((space_size, action_size)) / action_size

In [13]:
import time

start_time = time.time()
Pi = tabular_uniform_random_policy(S.shape[0], A.shape[0])

V = iterative_policy_evaluation(S, A, P, T, Pi)
print("--- %s seconds ---" % (time.time() - start_time))

for i in range(n * m):
    if i % 5 == 0 and i != 0:
        print("")
    print(round(V[i], 7), end=" ")
        


--- 0.11963510513305664 seconds ---
-0.2946619 -0.7075249 -0.561987 -0.5697107 -0.1915121 
-0.4831402 -0.9920751 -0.9935427 -0.5383428 -0.2041058 
-0.6654835 -0.8142405 -0.9119507 -0.4078543 -0.0948449 
-0.3814643 -0.7204939 -0.4590321 -0.1028326 0.2287259 
-0.1553739 -0.2463394 -0.1194729 0.2226305 0.1117107 

In [114]:
print(len(S))

25


-0.29476562 -0.29476562 -0.29476562 -0.29476562 -0.29476562 
-0.99230396 -0.99230396 -0.99230396 -0.99230396 -0.99230396 
-0.91217499 -0.91217499 -0.91217499 -0.91217499 -0.91217499 
-0.10295628 -0.10295628 -0.10295628 -0.10295628 -0.10295628 
0.11168043 0.11168043 0.11168043 0.11168043 0.11168043 


In [54]:
step(21, 0)

(20, 0.0, False)

In [82]:
 S.shape[0]

25

In [84]:
V = np.random.random((S.shape[0],))
V

array([0.26714842, 0.21421822, 0.32541552, 0.37578865, 0.41400078,
       0.55934601, 0.92963451, 0.1605333 , 0.02212195, 0.96035614,
       0.16957847, 0.66020583, 0.27217966, 0.08257213, 0.613618  ,
       0.37209803, 0.19578446, 0.48936394, 0.77395945, 0.47284574,
       0.37662938, 0.56220103, 0.24513202, 0.15666688, 0.21003998])

In [ ]:
def create_grid_world(w, h, reward_list, terminal):
    num_states = w * h

    S = np.arange(num_states)
    A = np.array([0, 1, 2, 3])  # 0: left, 1 : right, 2: top, 3: bot
    T = np.array(terminal)  # Terminal State ( position)
    P = np.zeros((len(S), len(A), len(S), 2))

    for x in range(w * h):
        if (x + 1) % w != 0:  # Droite
            P[x, 1, x + 1, 0] = 1.0
        if x % w != 0:  # Gauche
            P[x, 0, x - 1, 0] = 1.0
        if x >= w:  # Top
            P[x, 2, x - w, 0] = 1.0
        if x < w * h - w:  # Bot
            P[x, 3, x + w, 0] = 1.0

#     for r in reward_list:
#         set_reward(P, r, w, h)
    P[1, 1, 2, 1] = -1
    P[3, 0, 2, 1] = -1
    P[7, 2, 2, 1] = -1

    P[6, 3, 11, 1] = -1
    P[10, 1, 11, 1] = -1
    P[12, 0, 11, 1] = -1
    P[16, 2, 11, 1] = -1

    # P[13, 3, 18, 1] = -1
    # P[17, 1, 11, 1] = -1
    # P[19, 0, 18, 1] = -1
    # P[23, 2, 18, 1] = -1

    P[23, 1, 24, 1] = 1
    P[19, 3, 24, 1] = 1

    return S, A, T, P

def set_reward(P, reward_tuple, n, m):
    if (reward_tuple[0] + 1) % n != 0: # case de Droite
        P[reward_tuple[0] + 1, 0,reward_tuple[0], 1] = reward_tuple[1]

    if (reward_tuple[0] - 1) % n != 0: # case de Gauche
        P[reward_tuple[0] - 1, 1, reward_tuple[0], 1] = reward_tuple[1]

    if reward_tuple[0] - n >= n: #  case du top
        P[reward_tuple[0] - n, 3, reward_tuple[0], 1] = reward_tuple[1]

    if reward_tuple[0] + n < n * m - n: # case du bot
        P[reward_tuple[0] + n, 2, reward_tuple[0], 1] = reward_tuple[1]

def tabular_uniform_random_policy(space_size: int, action_size: int):
    return np.ones((space_size, action_size))

def reset(w, h) -> int:
    return w * h // 2

def is_terminal(state: int, T) -> bool:
    return state in T

In [ ]:
w = 5
h = 5
rewards = ((24, 1),
           (2, -1),
           (11, -1))

terminal = [2, 11, 24]

S, A, T, P = create_grid_world(w, h, rewards, terminal)
Pi = tabular_uniform_random_policy(S.shape[0], A.shape[0])

V = iterative_policy_evaluation(S, A, P, T, Pi)


for i in range(w * h):
    if i % 5 == 0 and i != 0:
        print("")
    print(round(V[i], 7), end=" ")

In [ ]:
w = 5
h = 5
rewards = ((24, 1),
           (2, -1),
           (11, -1))

terminal = [2, 11, 24]

S, A, T, P = create_grid_world(w, h, rewards, terminal)
Pi = tabular_uniform_random_policy(S.shape[0], A.shape[0])

V = iterative_policy_evaluation(S, A, P, T, Pi)


for i in range(w * h):
    if i % 5 == 0 and i != 0:
        print("")
    print(round(V[i], 7), end=" ")